# LIQA Transcript Quantification Results Analysis Part 1

In this notebook, I imported and merged transcript quantification results obtained from LIQA. The long-read RNA-seq dataset I used is from Glinos et al. (2022) paper. This notebook also includes a section for exploring missing output from LIQA.

## Part 1: Import Data and Configure Python Libraries

In [2]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec
%matplotlib inline
import seaborn as sns
import re
from IPython.display import display
from matplotlib.pyplot import gcf
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler
from PIL import ImageColor
from matplotlib.patches import Patch #for custom legend making
import scipy.spatial as sp, scipy.cluster.hierarchy as hc #for faster computing of hierarchial clusters

In [2]:
#pd.options.display.max_columns = None #display all columns in dataframe
#pd.options.display.max_rows = None

In [3]:
#pd.options.display.max_colwidth = 100 #show the full content of long strings

### Import Data

In [3]:
os.getcwd()

'C:\\Users\\15082\\OneDrive\\Desktop\\thesis_research\\gtex_v9_data_analysis\\LIQA\\work_in_progress'

In [4]:
data_dir = 'gtex_v9_data\\data_for_analysis'
liqa_quant_results_file_path = os.path.join(data_dir, 'my_liqa_data')
sample_info_path = os.path.join(data_dir, 'gtex_database_data\\sample_info_complete.csv')

#### Read data into pandas dataframe

In [6]:
# change working directory
os.chdir('C:\\Users\\15082\\OneDrive\\Desktop\\thesis_research')

In [7]:
sample_info = pd.read_csv(sample_info_path,sep=',')

## Part 2: Data Manipulation and Merging

### Merge Data (LIQA quantification output files)

Merge 92 separate files into one Pandas dataframe.

In [8]:
#change our working directory to the directory where we have all the data files
os.chdir(liqa_quant_results_file_path)

In [9]:
# Getting the FileNames of all txt files in the current dir.
filenames = [i for i in glob.glob(f"*.txt")] 

In [10]:
# base case to build the merged dataframe
liqa_merged_df = pd.read_csv(filenames[0],sep="\t")
liqa_merged_df['composite_id'] = liqa_merged_df.apply(lambda x:'%s_%s' % (x['IsoformName'],x['GeneName']),axis=1)

In [11]:
liqa_merged_df.head(3)

,GeneName,IsoformName,ReadPerGene_corrected,RelativeAbundance,infor_ratio,composite_id
0,ENSG00000204540.10,ENST00000479581.5,3.675106e-16,9.187765e-17,0.25,ENST00000479581.5_ENSG00000204540.10
1,ENSG00000204540.10,130750ff-7bd5-42fc-9951-2ee00e4c4253_ENSG00000...,1.072175e-16,2.680438e-17,0.25,130750ff-7bd5-42fc-9951-2ee00e4c4253_ENSG00000...
2,ENSG00000204540.10,0d6ef114-5d3f-4df1-a18b-02c128c21368_ENSG00000...,1.093079e-03,2.732699e-04,0.25,0d6ef114-5d3f-4df1-a18b-02c128c21368_ENSG00000...


In [12]:
liqa_merged_df.shape

(49189, 6)

In [14]:
liqa_merged_df.head()

,GeneName,IsoformName,ReadPerGene_corrected,RelativeAbundance,infor_ratio,composite_id
0,ENSG00000204540.10,ENST00000479581.5,3.675106e-16,9.187765e-17,0.25,ENST00000479581.5_ENSG00000204540.10
1,ENSG00000204540.10,130750ff-7bd5-42fc-9951-2ee00e4c4253_ENSG00000...,1.072175e-16,2.680438e-17,0.25,130750ff-7bd5-42fc-9951-2ee00e4c4253_ENSG00000...
2,ENSG00000204540.10,0d6ef114-5d3f-4df1-a18b-02c128c21368_ENSG00000...,1.093079e-03,2.732699e-04,0.25,0d6ef114-5d3f-4df1-a18b-02c128c21368_ENSG00000...
3,ENSG00000204540.10,e692f135-46a2-4cf8-953c-c0c636810214_ENSG00000...,1.560275e-16,3.900688e-17,0.25,e692f135-46a2-4cf8-953c-c0c636810214_ENSG00000...
4,ENSG00000204540.10,73af7ae0-6dc4-4d06-8132-8e644cb837f4_ENSG00000...,3.998907e+00,9.997267e-01,0.25,73af7ae0-6dc4-4d06-8132-8e644cb837f4_ENSG00000...


In [15]:
liqa_merged_df = liqa_merged_df[['composite_id','ReadPerGene_corrected']]
liqa_merged_df = liqa_merged_df.rename(columns={'ReadPerGene_corrected': filenames[0].split('.')[0]})

In [16]:
liqa_merged_df.head()

,composite_id,GTEX-1192X-0011-R10a-SM-4RXXZ
0,ENST00000479581.5_ENSG00000204540.10,3.675106e-16
1,130750ff-7bd5-42fc-9951-2ee00e4c4253_ENSG00000...,1.072175e-16
2,0d6ef114-5d3f-4df1-a18b-02c128c21368_ENSG00000...,1.093079e-03
3,e692f135-46a2-4cf8-953c-c0c636810214_ENSG00000...,1.560275e-16
4,73af7ae0-6dc4-4d06-8132-8e644cb837f4_ENSG00000...,3.998907e+00


In [17]:
# remove the first filename from the list
filenames_updated = filenames[1:] #remove the first file because it's already used to make the base dataframe

In [18]:
len(filenames_updated)

91

In [20]:
# loop through the list of filenames, create pandas dataframe, extract columns of interest and append to liqa_merged_df
for file in filenames_updated:
    # read text file into pandas dataframe
    quant_df = pd.read_csv(file,sep="\t")
    # create composite id
    quant_df['composite_id'] = quant_df.apply(lambda x:'%s_%s' % (x['IsoformName'],x['GeneName']),axis=1)
    # select columns
    quant_df = quant_df[['composite_id','ReadPerGene_corrected']]
    # rename column
    quant_df = quant_df.rename(columns={'ReadPerGene_corrected': file.split('.')[0]})
    # merge with liqa_merged_df
    liqa_merged_df = pd.merge(liqa_merged_df,quant_df,
                              left_on='composite_id',right_on='composite_id',how='outer')

In [21]:
liqa_merged_df.shape

(80371, 93)

In [22]:
liqa_merged_df.head()

,composite_id,GTEX-1192X-0011-R10a-SM-4RXXZ,GTEX-11H98-0011-R11b-SM-4SFLZ,GTEX-11TTK-0011-R7b-SM-4TVFS,GTEX-1211K-0826-SM-7LDFQ,GTEX-1313W-0011-R7b-SM-4ZL3U,GTEX-13QBU-0426-SM-5A4VT,GTEX-13QJ3-0726-SM-7LDHS,GTEX-13QJ3-0726-SM-7LDHS_rep,GTEX-13RTJ-0011-R7b-SM-5CTCB,...,GTEX-ZPU1-0826-SM-4UJSC,GTEX-ZT9X-0326-SM-4U9QG,GTEX-ZT9X-1826-SM-4V2KV,GTEX-ZT9X-1826-SM-4V2KV_rep,GTEX-ZT9X-1826-SM-4V2KV_rep2,GTEX-ZVZP-0226-SM-4VEIO,K562_ampure,K562_ampure_70ng,K562_extrawash,K562_extrawashwarm
0,ENST00000479581.5_ENSG00000204540.10,3.675106e-16,1.125000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.399547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,130750ff-7bd5-42fc-9951-2ee00e4c4253_ENSG00000...,1.072175e-16,0.916406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0d6ef114-5d3f-4df1-a18b-02c128c21368_ENSG00000...,1.093079e-03,0.003797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.852157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,e692f135-46a2-4cf8-953c-c0c636810214_ENSG00000...,1.560275e-16,1.333594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,73af7ae0-6dc4-4d06-8132-8e644cb837f4_ENSG00000...,3.998907e+00,5.621203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.747948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Export the Merged LIQA Quant Result Dataframe

In [36]:
# change working directory
os.chdir('C:\\Users\\15082\\OneDrive\\Desktop\\thesis_research')

In [37]:
liqa_merged_df.to_csv('gtex_v9_data\\data_for_analysis\\my_liqa_data\\liqa_merged.csv', sep=',')

### Check Missing Data

In [23]:
liqa_merged_df[liqa_merged_df.composite_id=='ENST00000368935.1_ENSG00000143363.15']

,composite_id,GTEX-1192X-0011-R10a-SM-4RXXZ,GTEX-11H98-0011-R11b-SM-4SFLZ,GTEX-11TTK-0011-R7b-SM-4TVFS,GTEX-1211K-0826-SM-7LDFQ,GTEX-1313W-0011-R7b-SM-4ZL3U,GTEX-13QBU-0426-SM-5A4VT,GTEX-13QJ3-0726-SM-7LDHS,GTEX-13QJ3-0726-SM-7LDHS_rep,GTEX-13RTJ-0011-R7b-SM-5CTCB,...,GTEX-ZPU1-0826-SM-4UJSC,GTEX-ZT9X-0326-SM-4U9QG,GTEX-ZT9X-1826-SM-4V2KV,GTEX-ZT9X-1826-SM-4V2KV_rep,GTEX-ZT9X-1826-SM-4V2KV_rep2,GTEX-ZVZP-0226-SM-4VEIO,K562_ampure,K562_ampure_70ng,K562_extrawash,K562_extrawashwarm
71005,ENST00000368935.1_ENSG00000143363.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000065,NaN,0.007266,0.00381,0.002107,NaN,1.123313e-08,7.378645e-10,NaN,NaN


In [24]:
liqa_merged_df[liqa_merged_df.composite_id=='ENST00000262126.8_ENSG00000101745.16']

,composite_id,GTEX-1192X-0011-R10a-SM-4RXXZ,GTEX-11H98-0011-R11b-SM-4SFLZ,GTEX-11TTK-0011-R7b-SM-4TVFS,GTEX-1211K-0826-SM-7LDFQ,GTEX-1313W-0011-R7b-SM-4ZL3U,GTEX-13QBU-0426-SM-5A4VT,GTEX-13QJ3-0726-SM-7LDHS,GTEX-13QJ3-0726-SM-7LDHS_rep,GTEX-13RTJ-0011-R7b-SM-5CTCB,...,GTEX-ZPU1-0826-SM-4UJSC,GTEX-ZT9X-0326-SM-4U9QG,GTEX-ZT9X-1826-SM-4V2KV,GTEX-ZT9X-1826-SM-4V2KV_rep,GTEX-ZT9X-1826-SM-4V2KV_rep2,GTEX-ZVZP-0226-SM-4VEIO,K562_ampure,K562_ampure_70ng,K562_extrawash,K562_extrawashwarm
80323,ENST00000262126.8_ENSG00000101745.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.929571


In [25]:
# Count the number of rows with missing values
num_rows_with_missing_values = liqa_merged_df.isnull().any(axis=1).sum()

In [26]:
print(num_rows_with_missing_values)

77273


In [27]:
check_missing_data_df = liqa_merged_df[['composite_id','GTEX-1192X-0011-R10a-SM-4RXXZ']]

In [28]:
sample1_missing_data_df = check_missing_data_df[check_missing_data_df['GTEX-1192X-0011-R10a-SM-4RXXZ'].isna()]

In [29]:
sample1_missing_data_df

,composite_id,GTEX-1192X-0011-R10a-SM-4RXXZ
49189,da83408b-dbbc-4b03-bd08-ae483c2fe37d_ENSG00000...,NaN
49190,c06d3500-df49-49de-b19a-239cd9eea69c_ENSG00000...,NaN
49191,9e3aae26-4af9-4ad8-9d4d-8f809ef549fb_ENSG00000...,NaN
49192,5cb21c98-daa8-4f43-843f-0cd997de6975_ENSG00000...,NaN
49193,182479da-8511-4244-9a19-1270a183a241_ENSG00000...,NaN
...,...,...
80366,74a320c6-b08c-48ab-b21f-fee3ad71669f_ENSG00000...,NaN
80367,dc0b68a1-95fe-4962-8993-51bf0f30da17_ENSG00000...,NaN
80368,e878980f-9ef6-4ad5-b3f8-47dbb38fd9de_ENSG00000...,NaN
80369,ENST00000304992.10_ENSG00000174231.16,NaN


In [30]:
sample1_missing_id = sample1_missing_data_df['composite_id']

In [31]:
sample1_missing_id

49189    da83408b-dbbc-4b03-bd08-ae483c2fe37d_ENSG00000...
49190    c06d3500-df49-49de-b19a-239cd9eea69c_ENSG00000...
49191    9e3aae26-4af9-4ad8-9d4d-8f809ef549fb_ENSG00000...
49192    5cb21c98-daa8-4f43-843f-0cd997de6975_ENSG00000...
49193    182479da-8511-4244-9a19-1270a183a241_ENSG00000...
                               ...                        
80366    74a320c6-b08c-48ab-b21f-fee3ad71669f_ENSG00000...
80367    dc0b68a1-95fe-4962-8993-51bf0f30da17_ENSG00000...
80368    e878980f-9ef6-4ad5-b3f8-47dbb38fd9de_ENSG00000...
80369                ENST00000304992.10_ENSG00000174231.16
80370    5d80bee1-439b-42d6-bc8f-f9dd2d7d6b9c_ENSG00000...
Name: composite_id, Length: 31182, dtype: object

In [32]:
sample1_missing_gene_id = sample1_missing_data_df.composite_id.apply(lambda x: x.split('_').pop()).drop_duplicates()

In [33]:
len(sample1_missing_gene_id)

3961

In [34]:
sample1_missing_gene_id

49189    ENSG00000115216.13
49207     ENSG00000152454.3
49209    ENSG00000196204.11
49211    ENSG00000135624.15
49238    ENSG00000099840.13
                ...        
80314    ENSG00000101745.16
80324    ENSG00000185627.17
80342     ENSG00000167985.6
80345    ENSG00000163938.16
80365    ENSG00000174231.16
Name: composite_id, Length: 3961, dtype: object

In [34]:
#sample1_missing_gene_id.to_csv('sample1_missing_gene_id.csv', index=False) 